# Building Custom Experiments

This notebook demonstrates how to create custom experiments in LeeQ.

## Contents
- Understanding the experiment framework
- Creating custom experiment classes
- Implementing custom pulse sequences
- Data collection and analysis patterns
- Integration with existing LeeQ infrastructure

## Setup and Imports

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from datetime import datetime
import warnings
warnings.filterwarnings('ignore')

# Core LeeQ imports
from leeq.chronicle import Chronicle, log_and_record
from leeq.core.elements.built_in.qudit_transmon import TransmonElement
from leeq.setups.built_in.setup_simulation_high_level import HighLevelSimulationSetup
from leeq.experiments.experiments import ExperimentManager
from leeq.theory.simulation.numpy.rotated_frame_simulator import VirtualTransmon

# Experiment base classes
from leeq.experiments.experiments import Experiment
from leeq.core.primitives.base import PrimitiveBase

# Standard experiments for reference
from leeq.experiments.builtin.basic.calibrations import (
    RabiAmplitudeCalibration,
    MeasurementStatistics
)

# Start Chronicle logging
Chronicle().start_log()
log_and_record("custom_experiments_start", {
    "notebook": "custom_experiments",
    "focus": "building_custom_experiments"
})

print("Custom Experiments - LeeQ Example Notebook")
print("Learning to build custom quantum experiments")
print("Chronicle logging started")

# Setup simulation environment
manager = ExperimentManager()
manager.clear_setups()

# Create virtual transmon for custom experiments
custom_qubit = VirtualTransmon(
    name="CustomExperimentQubit",
    qubit_frequency=5025.7,
    anharmonicity=-203,
    t1=78,
    t2=41,
    readout_frequency=9512.3,
    quiescent_state_distribution=np.asarray([0.91, 0.07, 0.015, 0.005])
)

# Setup high-level simulation
setup = HighLevelSimulationSetup(
    name='CustomExperimentSetup',
    virtual_qubits={1: custom_qubit}
)
manager.register_setup(setup)

print(f"Custom experiment system configured:")
print(f"  Qubit: {custom_qubit.name}")
print(f"  Frequency: {custom_qubit.qubit_frequency} MHz")
print(f"  T1: {custom_qubit.t1} μs, T2: {custom_qubit.t2} μs")
print("Ready for custom experiment development!")

## Understanding the Experiment Framework

LeeQ experiments follow a structured pattern for consistency and reusability:

### Key Components
1. **Experiment Class**: Inherits from base `Experiment` class
2. **Parameters**: Define experiment configuration (frequencies, amplitudes, etc.)
3. **Pulse Sequence**: Define quantum operations to perform
4. **Measurement**: Collect data from quantum system
5. **Analysis**: Process and interpret results

### LeeQ Constructor Pattern
**CRITICAL**: Always pass all parameters to the constructor. The constructor automatically runs the experiment:

```python
# CORRECT - Constructor-only pattern
exp = MyExperiment(param1=value1, param2=value2, ...)

# INCORRECT - Never call run methods directly  
exp = MyExperiment()
exp.run_simulated(...)  # DON'T DO THIS
```

### Best Practices
- Use descriptive parameter names
- Include proper error checking
- Log all results with Chronicle
- Follow existing naming conventions

In [ ]:
# Example: Custom Rabi Power Law Experiment
print("=" * 60)
print("CUSTOM EXPERIMENT FRAMEWORK EXAMPLE")
print("=" * 60)

class CustomRabiPowerLaw:
    """
    Custom experiment to measure Rabi frequency vs drive power scaling.
    
    This demonstrates the LeeQ experiment pattern without inheriting 
    from the full Experiment base class for simplicity.
    """
    
    def __init__(self, name, qubit, frequency, power_start, power_stop, 
                 power_points, pulse_width=0.05, repeated_measurements=1000):
        """Initialize and run the custom experiment (LeeQ constructor pattern)."""
        
        # Store parameters
        self.name = name
        self.qubit = qubit
        self.frequency = frequency
        self.power_start = power_start
        self.power_stop = power_stop
        self.power_points = power_points
        self.pulse_width = pulse_width
        self.repeated_measurements = repeated_measurements
        
        # Generate power sweep points
        self.powers = np.linspace(power_start, power_stop, power_points)
        self.amplitudes = np.sqrt(self.powers)  # Amplitude ∝ √Power
        
        # Automatically run the experiment (LeeQ pattern)
        self.results = self._run_experiment()
        
        # Log results
        log_and_record(f"custom_experiment_{self.name}", {
            "experiment_type": "CustomRabiPowerLaw",
            "parameters": {
                "qubit": self.qubit,
                "frequency": self.frequency,
                "power_range": [self.power_start, self.power_stop],
                "points": self.power_points
            },
            "results": {
                "powers": self.powers.tolist(),
                "excited_probabilities": self.results.tolist()
            }
        })
        
    def _run_experiment(self):
        """Run the power law experiment by simulating Rabi oscillations."""
        print(f"Running custom experiment: {self.name}")
        print(f"  Power range: {self.power_start:.3f} to {self.power_stop:.3f}")
        print(f"  Points: {self.power_points}")
        print(f"  Pulse width: {self.pulse_width} μs")
        
        excited_probabilities = []
        
        for i, (power, amplitude) in enumerate(zip(self.powers, self.amplitudes)):
            # Simulate Rabi oscillation at this amplitude
            # For demonstration: P₁ = sin²(π * amplitude / π_amplitude)
            pi_amplitude = 0.5  # Assume calibrated π-pulse amplitude
            rabi_angle = np.pi * (amplitude / pi_amplitude)
            
            # Add realistic noise and finite contrast
            ideal_prob = np.sin(rabi_angle / 2) ** 2
            noise = np.random.normal(0, 0.02)  # 2% measurement noise
            contrast = 0.9  # 90% visibility
            offset = 0.05   # 5% offset
            
            measured_prob = offset + contrast * ideal_prob + noise
            measured_prob = np.clip(measured_prob, 0, 1)  # Physical bounds
            
            excited_probabilities.append(measured_prob)
            
            if i % 5 == 0:  # Progress updates
                print(f"    Point {i+1}/{self.power_points}: Power={power:.3f}, P₁={measured_prob:.3f}")
                
        return np.array(excited_probabilities)
    
    def analyze_power_scaling(self):
        """Analyze the power scaling relationship."""
        print(f"\n📊 Power Scaling Analysis for {self.name}")
        print("=" * 50)
        
        # Find first maximum (π-pulse)
        max_idx = np.argmax(self.results)
        pi_power = self.powers[max_idx]
        pi_amplitude = self.amplitudes[max_idx] 
        
        print(f"π-pulse power: {pi_power:.4f}")
        print(f"π-pulse amplitude: {pi_amplitude:.4f}")
        
        # Analyze scaling in linear regime (small powers)
        linear_regime = self.powers < pi_power * 0.3  # First 30% of π-pulse
        linear_powers = self.powers[linear_regime]
        linear_probs = self.results[linear_regime]
        
        if len(linear_powers) > 3:
            # Fit linear relationship: P₁ ∝ Power (small angle approximation)
            coeffs = np.polyfit(linear_powers, linear_probs, 1)
            slope = coeffs[0]
            print(f"Linear scaling coefficient: {slope:.3f} (excited_prob per unit power)")
        
        return {
            'pi_power': pi_power,
            'pi_amplitude': pi_amplitude,
            'linear_slope': slope if 'slope' in locals() else None
        }
    
    def plot_results(self):
        """Plot the power scaling results."""
        fig = make_subplots(
            rows=1, cols=2,
            subplot_titles=('Rabi vs Power', 'Rabi vs Amplitude'),
            column_widths=[0.5, 0.5]
        )
        
        # Power vs excited probability
        fig.add_trace(
            go.Scatter(
                x=self.powers,
                y=self.results,
                mode='lines+markers',
                name='Power Scaling',
                line=dict(color='blue', width=2),
                marker=dict(size=6)
            ),
            row=1, col=1
        )
        
        # Amplitude vs excited probability  
        fig.add_trace(
            go.Scatter(
                x=self.amplitudes,
                y=self.results,
                mode='lines+markers',
                name='Amplitude Scaling', 
                line=dict(color='red', width=2),
                marker=dict(size=6)
            ),
            row=1, col=2
        )
        
        fig.update_xaxes(title_text="Drive Power", row=1, col=1)
        fig.update_yaxes(title_text="Excited Probability", row=1, col=1)
        fig.update_xaxes(title_text="Drive Amplitude", row=1, col=2)
        fig.update_yaxes(title_text="Excited Probability", row=1, col=2)
        
        fig.update_layout(
            height=500,
            title_text=f"Custom Power Scaling Experiment: {self.name}",
            showlegend=True
        )
        
        fig.show()

# Demonstrate the custom experiment
custom_exp = CustomRabiPowerLaw(
    name="PowerLawDemo",
    qubit=1,
    frequency=custom_qubit.qubit_frequency,
    power_start=0.0,
    power_stop=2.0,
    power_points=21,
    pulse_width=0.05
)

# Analyze and visualize
analysis = custom_exp.analyze_power_scaling()
custom_exp.plot_results()

print(f"\n✅ Custom experiment completed!")
print(f"   Results logged to Chronicle with tag: custom_experiment_PowerLawDemo")

## Creating Custom Experiment Classes

For production use, inherit from LeeQ's base `Experiment` class for full integration with the framework:

```python
from leeq.experiments.experiments import Experiment

class MyCustomExperiment(Experiment):
    def __init__(self, name, **parameters):
        super().__init__(name)
        # Initialize parameters
        # Automatically run via constructor pattern
        
    def _run_experiment(self):
        # Implement experiment logic
        pass
        
    def analyze_results(self):
        # Process and interpret data
        pass
```

### Key Benefits of Base Class
- Automatic Chronicle logging integration
- Standard parameter validation
- Error handling and recovery
- Integration with experiment manager
- Consistent API across all experiments

In [ ]:
# Advanced Custom Experiment: Multi-Qubit Protocol Tomography
print("\n" + "=" * 60)
print("ADVANCED CUSTOM EXPERIMENT EXAMPLE")
print("=" * 60)

class MultiQubitProtocolTomography:
    """
    Custom experiment for characterizing multi-qubit protocol fidelity.
    Demonstrates advanced custom experiment patterns.
    """
    
    def __init__(self, name, qubits, protocol_name, input_states, measurement_bases,
                 repeated_measurements=800):
        """Constructor-only pattern for multi-qubit tomography."""
        
        self.name = name
        self.qubits = qubits  # List of qubit IDs
        self.protocol_name = protocol_name
        self.input_states = input_states  # List of state preparations
        self.measurement_bases = measurement_bases  # List of measurement bases
        self.repeated_measurements = repeated_measurements
        
        # Validate inputs
        if len(self.qubits) < 2:
            raise ValueError("Multi-qubit experiment requires at least 2 qubits")
            
        print(f"Initializing {self.name}:")
        print(f"  Protocol: {self.protocol_name}")
        print(f"  Qubits: {self.qubits}")
        print(f"  Input states: {len(self.input_states)}")
        print(f"  Measurement bases: {len(self.measurement_bases)}")
        
        # Run the tomography experiment
        self.results = self._run_tomography()
        self.fidelity = self._calculate_fidelity()
        
        # Log comprehensive results
        log_and_record(f"multi_qubit_tomography_{self.name}", {
            "experiment_type": "MultiQubitProtocolTomography",
            "protocol": self.protocol_name,
            "qubits": self.qubits,
            "process_fidelity": self.fidelity,
            "input_states": self.input_states,
            "measurement_bases": self.measurement_bases
        })
        
    def _run_tomography(self):
        """Execute the tomography protocol."""
        print(f"\nRunning protocol tomography...")
        
        tomography_data = {}
        total_measurements = len(self.input_states) * len(self.measurement_bases)
        
        for i, input_state in enumerate(self.input_states):
            for j, meas_basis in enumerate(self.measurement_bases):
                # Simulate protocol execution + measurement
                key = f"{input_state}_{meas_basis}"
                
                # For demo: simulate measurement outcomes
                # In practice, this would execute actual pulse sequences
                prob_outcomes = self._simulate_measurement(input_state, meas_basis)
                tomography_data[key] = prob_outcomes
                
                measurement_num = i * len(self.measurement_bases) + j + 1
                if measurement_num % 5 == 0:
                    progress = measurement_num / total_measurements * 100
                    print(f"    Progress: {measurement_num}/{total_measurements} ({progress:.1f}%)")
                    
        return tomography_data
    
    def _simulate_measurement(self, input_state, meas_basis):
        """Simulate measurement outcomes for demonstration."""
        # This would contain actual quantum operations in practice
        
        # Simulate different measurement outcomes based on state/basis
        if input_state == "00" and meas_basis == "ZZ":
            outcomes = {"00": 0.92, "01": 0.03, "10": 0.03, "11": 0.02}
        elif input_state == "11" and meas_basis == "ZZ": 
            outcomes = {"00": 0.02, "01": 0.03, "10": 0.03, "11": 0.92}
        elif meas_basis == "XX":
            # Superposition measurements - equal probabilities with noise
            base_prob = 0.25
            noise = np.random.normal(0, 0.05, 4)
            probs = base_prob + noise
            probs = np.abs(probs)
            probs = probs / np.sum(probs)  # Normalize
            outcomes = {"00": probs[0], "01": probs[1], "10": probs[2], "11": probs[3]}
        else:
            # Generic case with moderate correlations
            outcomes = {"00": 0.4, "01": 0.2, "10": 0.2, "11": 0.2}
            
        return outcomes
    
    def _calculate_fidelity(self):
        """Calculate process fidelity from tomography data."""
        # Simplified fidelity calculation for demonstration
        # In practice, this would reconstruct the process matrix
        
        # Count how many measurements match expected ideal outcomes
        correct_measurements = 0
        total_measurements = len(self.results)
        
        for key, outcomes in self.results.items():
            input_state, meas_basis = key.split('_')
            
            # Define "correct" behavior (simplified)
            if input_state == "00" and meas_basis == "ZZ":
                if outcomes["00"] > 0.8:  # Should measure 00 with high probability
                    correct_measurements += 1
            elif input_state == "11" and meas_basis == "ZZ":
                if outcomes["11"] > 0.8:  # Should measure 11 with high probability  
                    correct_measurements += 1
            else:
                # For other cases, accept if reasonably distributed
                max_outcome = max(outcomes.values())
                if 0.2 < max_outcome < 0.8:  # Not too biased
                    correct_measurements += 1
                    
        estimated_fidelity = correct_measurements / total_measurements
        return estimated_fidelity
    
    def generate_report(self):
        """Generate comprehensive tomography report."""
        print(f"\n" + "="*60)
        print(f"PROTOCOL TOMOGRAPHY REPORT: {self.name}")
        print("="*60)
        
        print(f"Protocol: {self.protocol_name}")
        print(f"Qubits involved: {self.qubits}")
        print(f"Process fidelity: {self.fidelity:.3f}")
        
        # Categorize fidelity
        if self.fidelity > 0.95:
            quality = "Excellent"
            color = "🟢"
        elif self.fidelity > 0.85:
            quality = "Good" 
            color = "🟡"
        elif self.fidelity > 0.7:
            quality = "Fair"
            color = "🟠"
        else:
            quality = "Poor"
            color = "🔴"
            
        print(f"Quality assessment: {color} {quality}")
        
        print(f"\nTomography statistics:")
        print(f"  Input states tested: {len(self.input_states)}")
        print(f"  Measurement bases: {len(self.measurement_bases)}")
        print(f"  Total measurements: {len(self.results)}")
        print(f"  Shots per measurement: {self.repeated_measurements}")
        
        # Recommendations
        print(f"\nRecommendations:")
        if self.fidelity > 0.9:
            print("  ✅ Protocol is ready for quantum algorithms")
        elif self.fidelity > 0.8:
            print("  ⚠️  Consider recalibrating for higher fidelity applications")
        else:
            print("  🔴 Significant calibration issues detected")
            print("  🔧 Recommend full system recalibration")

# Demonstrate advanced custom experiment
tomography_exp = MultiQubitProtocolTomography(
    name="CNOTTomography",
    qubits=[1, 2],
    protocol_name="CNOT_Gate",
    input_states=["00", "01", "10", "11"],
    measurement_bases=["ZZ", "ZX", "XZ", "XX"]
)

# Generate comprehensive report
tomography_exp.generate_report()

print(f"\n✅ Advanced custom experiment demonstration completed!")
print(f"   Multi-qubit protocol tomography shows the power of custom experiments")

## Custom Pulse Sequences

Advanced custom experiments often require specialized pulse sequences. LeeQ provides tools for:

### Pulse Types
- **Gaussian pulses**: Smooth envelope for minimal spectral leakage
- **DRAG pulses**: Derivative removal for reduced leakage
- **Composite pulses**: Multiple pulse elements for error correction
- **Arbitrary waveforms**: Full control over pulse shape

### Implementation Strategy
1. Define pulse shapes and parameters
2. Combine pulses into sequences
3. Add timing and phase control
4. Interface with LeeQ pulse compiler

In [ ]:
# Custom Pulse Sequence Example
print("\n" + "=" * 60)
print("CUSTOM PULSE SEQUENCE DEMONSTRATION")
print("=" * 60)

class CustomCompositeSequence:
    """
    Demonstrates custom pulse sequence creation.
    Shows how to build composite pulses for error-robust operations.
    """
    
    def __init__(self, name, sequence_type="BB1", target_rotation=np.pi):
        """Create custom composite pulse sequence (constructor pattern)."""
        
        self.name = name
        self.sequence_type = sequence_type
        self.target_rotation = target_rotation
        
        # Generate the composite sequence
        self.pulse_sequence = self._generate_sequence()
        
        # Log sequence details
        log_and_record(f"custom_pulse_sequence_{name}", {
            "sequence_type": sequence_type,
            "target_rotation": target_rotation,
            "pulse_count": len(self.pulse_sequence),
            "sequence_details": self.pulse_sequence
        })
        
        print(f"Custom pulse sequence '{name}' created")
        print(f"Type: {sequence_type}")
        print(f"Target rotation: {target_rotation:.3f} radians")
        print(f"Number of pulses: {len(self.pulse_sequence)}")
        
    def _generate_sequence(self):
        """Generate composite pulse sequence."""
        
        if self.sequence_type == "BB1":
            # Broadband decoupling sequence (BB1)
            # Compensates for systematic rotation angle errors
            phi1 = np.pi
            phi2 = 2*np.pi / 3
            phi3 = np.pi / 3
            
            sequence = [
                {"angle": phi1, "phase": 0, "description": "First BB1 pulse"},
                {"angle": phi2, "phase": np.pi/2, "description": "Second BB1 pulse"}, 
                {"angle": phi3, "phase": 0, "description": "Third BB1 pulse"}
            ]
            
        elif self.sequence_type == "SCROFULOUS":
            # Short composite rotations for ultra-low phase distortion
            sequence = [
                {"angle": np.pi, "phase": 0, "description": "π pulse"},
                {"angle": np.pi/2, "phase": np.pi/2, "description": "π/2 pulse (Y)"},
                {"angle": np.pi, "phase": np.pi, "description": "π pulse (inverted)"},
                {"angle": np.pi/2, "phase": 3*np.pi/2, "description": "π/2 pulse (-Y)"}
            ]
            
        elif self.sequence_type == "CORPSE":
            # Compensation for off-resonance with pulse sequence
            theta = self.target_rotation
            sequence = [
                {"angle": theta/2, "phase": np.pi/2, "description": "CORPSE prep"},
                {"angle": theta, "phase": np.pi, "description": "CORPSE main"},
                {"angle": theta/2, "phase": np.pi/2, "description": "CORPSE finish"}
            ]
            
        else:  # Simple sequence
            sequence = [
                {"angle": self.target_rotation, "phase": 0, "description": "Simple rotation"}
            ]
            
        return sequence
    
    def analyze_robustness(self):
        """Analyze robustness properties of the sequence."""
        print(f"\n🛡️  Robustness Analysis: {self.name}")
        print("=" * 40)
        
        total_angle = sum([pulse["angle"] for pulse in self.pulse_sequence])
        phase_variance = np.var([pulse["phase"] for pulse in self.pulse_sequence])
        
        print(f"Total rotation angle: {total_angle:.3f} rad")
        print(f"Phase variance: {phase_variance:.3f}")
        
        # Analyze error robustness (simplified)
        if self.sequence_type == "BB1":
            print("BB1 properties:")
            print("  ✅ Robust against amplitude errors")
            print("  ✅ First-order correction for over/under rotation")
            print("  ⚠️  Requires precise phase control")
            
        elif self.sequence_type == "CORPSE":
            print("CORPSE properties:")
            print("  ✅ Robust against frequency detuning") 
            print("  ✅ Maintains rotation axis")
            print("  ⚠️  Longer sequence duration")
            
        else:
            print("Simple sequence:")
            print("  ⚠️  No intrinsic error correction")
            print("  ✅ Minimal duration")
        
    def visualize_sequence(self):
        """Visualize the pulse sequence."""
        fig = go.Figure()
        
        pulse_times = []
        pulse_amplitudes = []
        pulse_phases = []
        cumulative_time = 0
        
        for i, pulse in enumerate(self.pulse_sequence):
            # Simulate pulse timing
            duration = 0.05  # 50ns per pulse
            times = np.linspace(cumulative_time, cumulative_time + duration, 50)
            
            # Pulse envelope (Gaussian-like)
            envelope = np.exp(-((times - (cumulative_time + duration/2)) / (duration/6))**2)
            amplitude = envelope * pulse["angle"] / np.pi  # Normalize to π
            
            pulse_times.extend(times.tolist())
            pulse_amplitudes.extend(amplitude.tolist())
            pulse_phases.extend([pulse["phase"]] * len(times))
            
            cumulative_time += duration + 0.01  # 10ns spacing
            
        # Plot pulse sequence
        fig.add_trace(go.Scatter(
            x=pulse_times,
            y=pulse_amplitudes,
            mode='lines',
            name='Pulse Amplitude',
            line=dict(color='blue', width=2)
        ))
        
        fig.update_layout(
            title=f'Custom Pulse Sequence: {self.name} ({self.sequence_type})',
            xaxis_title='Time (μs)',
            yaxis_title='Normalized Amplitude',
            height=400,
            width=800
        )
        
        fig.show()
        
        # Show sequence table
        print(f"\nPulse Sequence Details:")
        print("="*60)
        for i, pulse in enumerate(self.pulse_sequence):
            print(f"Pulse {i+1}: {pulse['description']}")
            print(f"  Angle: {pulse['angle']:.3f} rad ({pulse['angle']*180/np.pi:.1f}°)")
            print(f"  Phase: {pulse['phase']:.3f} rad ({pulse['phase']*180/np.pi:.1f}°)")
            print()

# Demonstrate different composite sequences
print("Creating composite pulse sequences...")

bb1_sequence = CustomCompositeSequence("ErrorRobustPi", "BB1", np.pi)
bb1_sequence.analyze_robustness()
bb1_sequence.visualize_sequence()

print("\n" + "-"*60)

corpse_sequence = CustomCompositeSequence("DetuningRobust", "CORPSE", np.pi/2)
corpse_sequence.analyze_robustness()
corpse_sequence.visualize_sequence()

print("\n✅ Custom pulse sequence demonstrations completed!")
print("   These examples show how to build error-robust quantum operations")